In [61]:
import polars as pl


from excel_manager import (
    get_df,
    get_count_df,
    COL_NAME_TOTAL_COUNT,
    COL_NAME_WINDOW_TIME,
    COL_NAME_DEPARTURE_DATETIME,
)

In [62]:
COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY = "flight_with_delay"
COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_GTE_15MIN = "flight_with_delay_gte_15min"
COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_41_42_GTE_15MIN = (
    "flight_with_delay_gte_15min_code_41_42"
)
COL_NAME_PER_DELAYED_FLIGHTS_15MIN_NOT_WITH_41_42 = (
    "per_delayed_flights_15min__not_with_41_42"
)
COL_NAME_PER_DELAYED_FLIGHTS_NOT_WITH_15MIN = "per_delayed_flights_not_with_15min"
COL_NAME_PER_FLIGHTS_NOT_DELAYED = "per_flights_not_delayed"

In [63]:
df = get_df()

In [64]:
window_str = ""

In [65]:
total_df = get_count_df(window_str)

In [66]:
total_df.collect()

total_count
u32
17017


In [67]:
windowed_df = df.with_columns(
    pl.col(COL_NAME_DEPARTURE_DATETIME)
    .dt.truncate(window_str)
    .alias(COL_NAME_WINDOW_TIME)
)
delayed_flights_count_df = windowed_df.group_by(COL_NAME_WINDOW_TIME).agg(
    pl.len().alias(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY)
)

In [68]:
delayed_flights_count_df.sort("WINDOW_DATETIME_DEP").collect()

PanicException: attempt to calculate the remainder with a divisor of zero

In [69]:
delayed_15min_df = windowed_df.filter((pl.col("Retard en min") >= 15))
delayed_15min_count_df = delayed_15min_df.group_by(COL_NAME_WINDOW_TIME).agg(
    pl.len().alias(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_GTE_15MIN)
)

In [70]:
delayed_15min_count_df.sort("WINDOW_DATETIME_DEP").collect()

PanicException: attempt to calculate the remainder with a divisor of zero

In [11]:
delayed_flights_41_42_gte_15min_count_df = (
    delayed_15min_df.filter(pl.col("CODE_DR").is_in({41, 42}))
    .group_by(COL_NAME_WINDOW_TIME)
    .agg(pl.len().alias(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_41_42_GTE_15MIN))
)

In [12]:
delayed_flights_41_42_gte_15min_count_df.sort("WINDOW_DATETIME_DEP").collect()

WINDOW_DATETIME_DEP,flight_with_delay_gte_15min_code_41_42
datetime[μs],u32
2025-03-30 00:00:00,2
2025-03-31 00:00:00,2
2025-04-01 00:00:00,2
2025-04-02 00:00:00,2
2025-04-03 00:00:00,3
…,…
2025-06-20 00:00:00,1
2025-06-21 00:00:00,1
2025-06-22 00:00:00,5


In [13]:
joined_df = (
    total_df.join(delayed_flights_count_df, COL_NAME_WINDOW_TIME, how="left")
    .join(delayed_15min_count_df, COL_NAME_WINDOW_TIME, how="left")
    .join(delayed_flights_41_42_gte_15min_count_df, COL_NAME_WINDOW_TIME, how="left")
    .fill_null(0)
)

In [14]:
joined_df.collect()

WINDOW_DATETIME_DEP,total_count,flight_with_delay,flight_with_delay_gte_15min,flight_with_delay_gte_15min_code_41_42
datetime[μs],u32,u32,u32,u32
2025-05-09 00:00:00,209,5,4,3
2025-04-21 00:00:00,200,4,4,3
2025-06-07 00:00:00,194,6,5,4
2025-04-03 00:00:00,199,3,3,3
2025-05-20 00:00:00,167,4,3,1
…,…,…,…,…
2025-05-27 00:00:00,154,5,5,4
2025-05-16 00:00:00,214,7,6,3
2025-04-28 00:00:00,191,1,0,0


In [15]:
joined_df_res = joined_df.with_columns(
    [
        ## delay
        pl.lit(1)
        .sub(
            pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY)
            / (pl.col(COL_NAME_TOTAL_COUNT))
        )
        .alias(COL_NAME_PER_FLIGHTS_NOT_DELAYED),
        ## delay > 15
        (
            pl.lit(1).sub(
                pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_GTE_15MIN)
                / (pl.col(COL_NAME_TOTAL_COUNT))
            )
        ).alias(COL_NAME_PER_DELAYED_FLIGHTS_NOT_WITH_15MIN),
        ## delay > 15 min for 41 42
        pl.lit(1)
        .sub(
            (
                pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_41_42_GTE_15MIN)
                / pl.col(COL_NAME_TOTAL_COUNT)
            )
        )
        .alias(COL_NAME_PER_DELAYED_FLIGHTS_15MIN_NOT_WITH_41_42),
    ]
)

In [16]:
joined_df_res = joined_df.sort(COL_NAME_WINDOW_TIME)

In [29]:
joined_df_res.collect()

WINDOW_DATETIME_DEP,total_count,flight_with_delay,flight_with_delay_gte_15min,flight_with_delay_gte_15min_code_41_42
datetime[μs],u32,u32,u32,u32
2025-03-30 00:00:00,197,2,2,2
2025-03-31 00:00:00,190,3,3,2
2025-04-01 00:00:00,161,4,2,2
2025-04-02 00:00:00,193,5,4,2
2025-04-03 00:00:00,199,3,3,3
…,…,…,…,…
2025-06-20 00:00:00,217,7,4,1
2025-06-21 00:00:00,204,1,1,1
2025-06-22 00:00:00,225,6,6,5


In [34]:
latest_values = joined_df_res.collect().select(pl.col(COL_NAME_TOTAL_COUNT).tail(2))

In [41]:
latest_values

total_count
u32
215
194


In [39]:
year, before = latest_values.to_series().to_list()

In [40]:
year

215

In [18]:
joined_df_res_2 = joined_df.with_columns(
    [
        ## delay
        (
            (
                pl.col(COL_NAME_TOTAL_COUNT)
                - pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY)
            ).cast(pl.Float64)
            / pl.col(COL_NAME_TOTAL_COUNT)
        ).alias(COL_NAME_PER_FLIGHTS_NOT_DELAYED),
        ## delay > 15
        (
            (
                pl.col(COL_NAME_TOTAL_COUNT)
                - pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_GTE_15MIN)
            )
            / pl.col(COL_NAME_TOTAL_COUNT)
        ).alias(COL_NAME_PER_DELAYED_FLIGHTS_NOT_WITH_15MIN),
        ## delay > 15 min for 41 42
        (
            (
                pl.col(COL_NAME_TOTAL_COUNT)
                - pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_41_42_GTE_15MIN)
            )
            / pl.col(COL_NAME_TOTAL_COUNT)
        ).alias(COL_NAME_PER_DELAYED_FLIGHTS_15MIN_NOT_WITH_41_42),
    ]
)

In [19]:
joined_df_res_2.collect()

WINDOW_DATETIME_DEP,total_count,flight_with_delay,flight_with_delay_gte_15min,flight_with_delay_gte_15min_code_41_42,per_flights_not_delayed,per_delayed_flights_not_with_15min,per_delayed_flights_15min__not_with_41_42
datetime[μs],u32,u32,u32,u32,f64,f64,f64
2025-06-07 00:00:00,194,6,5,4,0.969072,0.974227,0.979381
2025-04-21 00:00:00,200,4,4,3,0.98,0.98,0.985
2025-04-03 00:00:00,199,3,3,3,0.984925,0.984925,0.984925
2025-05-09 00:00:00,209,5,4,3,0.976077,0.980861,0.985646
2025-05-20 00:00:00,167,4,3,1,0.976048,0.982036,0.994012
…,…,…,…,…,…,…,…
2025-05-27 00:00:00,154,5,5,4,0.967532,0.967532,0.974026
2025-06-14 00:00:00,205,4,3,3,0.980488,0.985366,0.985366
2025-04-28 00:00:00,191,1,0,0,0.994764,1.0,1.0


In [20]:
joined_df_res_2.select(COL_NAME_WINDOW_TIME).collect().to_series().to_list()

[datetime.datetime(2025, 5, 20, 0, 0),
 datetime.datetime(2025, 4, 3, 0, 0),
 datetime.datetime(2025, 5, 9, 0, 0),
 datetime.datetime(2025, 4, 21, 0, 0),
 datetime.datetime(2025, 6, 7, 0, 0),
 datetime.datetime(2025, 5, 13, 0, 0),
 datetime.datetime(2025, 4, 14, 0, 0),
 datetime.datetime(2025, 6, 18, 0, 0),
 datetime.datetime(2025, 5, 2, 0, 0),
 datetime.datetime(2025, 5, 31, 0, 0),
 datetime.datetime(2025, 6, 11, 0, 0),
 datetime.datetime(2025, 4, 7, 0, 0),
 datetime.datetime(2025, 5, 24, 0, 0),
 datetime.datetime(2025, 4, 25, 0, 0),
 datetime.datetime(2025, 5, 6, 0, 0),
 datetime.datetime(2025, 3, 31, 0, 0),
 datetime.datetime(2025, 4, 18, 0, 0),
 datetime.datetime(2025, 4, 29, 0, 0),
 datetime.datetime(2025, 5, 17, 0, 0),
 datetime.datetime(2025, 6, 4, 0, 0),
 datetime.datetime(2025, 6, 22, 0, 0),
 datetime.datetime(2025, 6, 15, 0, 0),
 datetime.datetime(2025, 4, 22, 0, 0),
 datetime.datetime(2025, 4, 11, 0, 0),
 datetime.datetime(2025, 5, 10, 0, 0),
 datetime.datetime(2025, 5, 28, 

In [21]:
joined_df.collect()

WINDOW_DATETIME_DEP,total_count,flight_with_delay,flight_with_delay_gte_15min,flight_with_delay_gte_15min_code_41_42
datetime[μs],u32,u32,u32,u32
2025-04-03 00:00:00,199,3,3,3
2025-05-09 00:00:00,209,5,4,3
2025-05-20 00:00:00,167,4,3,1
2025-06-07 00:00:00,194,6,5,4
2025-04-21 00:00:00,200,4,4,3
…,…,…,…,…
2025-05-16 00:00:00,214,7,6,3
2025-05-27 00:00:00,154,5,5,4
2025-04-10 00:00:00,196,2,2,1


# normal


In [53]:
delayed_flights_count_df = df.select(
    pl.len().alias(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY)
)

delayed_15min_df = df.filter((pl.col("Retard en min") >= 15))
delayed_15min_count_df = delayed_15min_df.select(
    pl.len().alias(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_GTE_15MIN)
)

In [58]:
total_df.collect()

total_count
u32
17017


In [57]:
delayed_flights_count_df.collect()

flight_with_delay
u32
365


In [59]:
joined_df = pl.concat(
    [
        total_df,
        delayed_flights_count_df,
        delayed_15min_count_df,
        delayed_flights_41_42_gte_15min_count_df,
    ],
    how="horizontal",
)

In [60]:
joined_df.collect()

total_count,flight_with_delay,flight_with_delay_gte_15min,flight_with_delay_gte_15min_code_41_42
u32,u32,u32,u32
17017,365,294,194


In [24]:
count_flight = joined_df.select(pl.col(COL_NAME_TOTAL_COUNT)).item()
count_flight_with_delay = joined_df.select(
    pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY)
).item()
count_flight_with_delay_gte_15min = joined_df.select(
    pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_GTE_15MIN)
).item()
count_flight_with_delay_gte_15min_41_42 = joined_df.select(
    pl.col(COL_NAME_TOTAL_COUNT_FLIGHT_WITH_DELAY_41_42_GTE_15MIN)
).item()

AttributeError: 'LazyFrame' object has no attribute 'item'

In [ ]:
# first graph

per_delayed_flights_15min__not_with_41_42 = 1 - (
    count_flight_with_delay_gte_15min_41_42 / count_flight
)

# second graph

## first op (15 min)
per_delayed_flights_not_with_15min = 1 - (
    count_flight_with_delay_gte_15min / count_flight
)


## second op

per_flights_not_delayed = 1 - (count_flight_with_delay / count_flight)


# third graph

In [ ]:
import polars as pl
from datetime import timedelta


df_window = pl.col("DateTime").dt.truncate(timedelta(weeks=1))

df_windowed = df.group_by(df_window).agg(pl.len())